In [1]:
import pandas as pd
import nltk
import string
import numpy as np

In [2]:
articles = []

for l in range(1,4):
    k = 32
    if (l == 2):
        k=30     
    for i in range(1,k):
        dataP = pd.read_csv(str(l) + "_" + str(i) + "_P.csv")
        dataTH = pd.read_csv(str(l) + "_" + str(i) + "_TH.csv")
        dataTOI = pd.read_csv(str(l) + "_" + str(i) + "_TOI.csv")
        for j in range(len(dataP)):
            articles.append(dataP['Article'][j])
        for j in range(len(dataTH)):
            articles.append(dataTH['Content'][j])
        for j in range(len(dataTOI)):
            articles.append(dataTOI['Content'][j])
                     
     

In [ ]:
print(len(articles))

In [4]:
for i in range(0,len(articles)):
    if type(articles[i]) is not str:
        del articles[i]
    articles[i] = articles[i].lower()
   
    

In [5]:
for i in range(len(articles)):
    articles[i] = articles[i].translate(str.maketrans('','' , string.punctuation))

In [6]:
for i in range(len(articles)):
    articles[i] = " ".join(articles[i].split())

In [7]:
from nltk import word_tokenize
words = []


In [8]:
for i in range(len(articles)):
    words.append(word_tokenize(articles[i]))

In [9]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))


In [10]:
for i in range(len(words)):
    words[i] = [word for word in words[i] if word not in stop_words]  

In [11]:
for i in range(len(words)):
    words[i] = [token for token in words[i] if not token.isnumeric()] 

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(j) for j in words[i]]

In [13]:
for i in range(len(words)): 
    words[i] = [token for token in words[i] if len(token) > 1]

In [14]:
from gensim.models import Phrases

In [15]:
bigram = Phrases(words, min_count=5,threshold= 100)

In [16]:
for idx in range(len(words)):
    for token in bigram[words[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            words[idx].append(token)    



In [17]:
from gensim.corpora import Dictionary

In [18]:
dictionary = Dictionary(words)
dictionary.filter_extremes(no_below =5000 ,no_above=0.3, keep_n = 100000)


In [29]:
print(len(dictionary))

423


In [19]:
corpus = [dictionary.doc2bow(doc) for doc in words]


In [20]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [21]:
from gensim.models import LdaModel
num_topics = 6
chunksize = 5000
passes = 20
iterations = 400
eval_every = None

In [22]:
temp = dictionary[0]

In [28]:
id2word = dictionary.id2token
print(len(id2word))

423


In [24]:
model = LdaModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus)
for t in top_topics:
    print(t)
    print("\n")

In [25]:
import pyLDAvis.gensim

In [26]:
lda_display = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary, sort_topics=False)


C:\Users\ajain\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [27]:
pyLDAvis.display(lda_display)


In [ ]:
all_topics = model.get_document_topics(corpus_tfidf, per_word_topics=False)


In [ ]:
for i in range(0,10):
    print('New Document \n')
    print('Document topics:', all_topics[i])
    print(" ")
    print('-------------- \n')

In [36]:
from scipy.spatial import distance

In [ ]:
distance.jensenshannon(doc1,doc2)

In [106]:
from gensim.test.utils import datapath
    
temp_file = datapath("model2")
model.save((temp_file))

model2 = LdaModel.load(temp_file)

In [42]:
print(temp_file)
#model_mar = LdaModel.load(temp_file)

C:\Users\ajain\Anaconda3\lib\site-packages\gensim\test\test_data\modell


In [ ]:
from gensim.models import hdpmodel
hdp_model = hdpmodel.HdpModel(corpus=corpus, id2word=id2word)
hdp_model.print_topics(num_topics=-1)

In [ ]:
shown_topics = hdp_model.show_topics(num_topics=hdp_model.m_T, formatted=False)
topics_nos = [x[0] for x in shown_topics ]
weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]
ll= pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [ ]:
for i in range(0,150):
    print(ll['weight'][i])
    print('\n')

In [45]:
train_vecs = []
for i in range(len(words)):
    top_topics = model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    train_vecs.append(topic_vec)

In [46]:
print(train_vecs[0])

[6.304063e-05, 5.930225e-05, 0.19166397, 4.6081892e-05, 0.80810606, 6.153711e-05]


In [49]:
test_articles = []

dataP = pd.read_csv("4_1_P.csv")
dataTH = pd.read_csv("4_1_TH.csv")
dataTOI = pd.read_csv("4_1_TOI.csv")
for j in range(len(dataP)):
    test_articles.append(dataP['Article'][j])
for j in range(len(dataTH)):
    test_articles.append(dataTH['Content'][j])
for j in range(len(dataTOI)):
    test_articles.append(dataTOI['Content'][j])
                     
 

In [50]:
print(len(test_articles))

1158


In [52]:
for i in range(len(test_articles)):
    if type(test_articles[i]) is not str:
        del test_articles[i]
    test_articles[i] = test_articles[i].lower()

In [53]:
for i in range(len(test_articles)):
    test_articles[i] = test_articles[i].translate(str.maketrans('','' , string.punctuation))

In [54]:
for i in range(len(test_articles)):
    test_articles[i] = " ".join(test_articles[i].split())

In [55]:
test = []
for i in range(len(test_articles)):
    test.append(word_tokenize(test_articles[i]))

In [56]:
stop_words = set(stopwords.words("english"))
for i in range(len(test)):
    test[i] = [word for word in test[i] if word not in stop_words]
for i in range(len(test)):
    test[i] = [token for token in test[i] if not token.isnumeric()]
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
for i in range(len(test)):
    test[i] = [lemmatizer.lemmatize(j) for j in test[i]]
for i in range(len(test)):    
    test[i] = [token for token in test[i] if len(token) > 1]

In [57]:
testcorpus = [dictionary.doc2bow(doc) for doc in test]


In [61]:
test_vecs = []
for i in range(len(test)):
    top_topics = model.get_document_topics(testcorpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    test_vecs.append(topic_vec)

In [93]:
topic0 = []
topic1 = []
topic2 = []
topic3 = []
topic4 = []
topic5 = []
matched_topic = [topic0, topic1, topic2, topic3, topic4, topic5]


In [94]:
for i in range(len(test_vecs)):
    mx = -1
    for j in range(len(test_vecs[i])):
        mx = max(mx, test_vecs[i][j])
    
    for j in range(len(test_vecs[i])):
        if mx==test_vecs[i][j]:
            matched_topic[j].append(test_articles[i])
            break

In [ ]:
print((matched_topic[0][0]))

In [74]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""

    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """

    # Your Code Here
    token = nltk.word_tokenize(doc)
    # add parts of speech to token
    tag = nltk.pos_tag(token)
    # convert nltk pos into wordnet pos
    nltk2wordnet = [(i[0], convert_tag(i[1])) for i in tag]
    # if there are no synsets in token, ignore, else put in a list
    output = [wn.synsets(i, z)[0] for i, z in nltk2wordnet if len(wn.synsets(i, z))>0]

    return output


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """


    # Your Code Here
    list1 = []
    # For each synset in s1
    for a in s1:
        # finds the synset in s2 with the largest similarity value
        list1.append(max([i.path_similarity(a) for i in s2 if i.path_similarity(a) is not None]))

    output = sum(list1)/len(list1)

    return output


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""
            # first function u need to create
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
            # 2nd function u need to create
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2






In [ ]:
t = np.zeros((len(matched_topic[0]),len(matched_topic[0])))
t             

In [ ]:
for i in range(len(matched_topic[0])):
              doc1 = test_articles[matched_topic[0][i]]
              for j in range(i, len(matched_topic[0])):
                  doc2 = test_articles[matched_topic[0][j]]
                  if i == j:
                      t[i][j]= -1
                      continue
                  t[i][j] = document_path_similarity(doc1, doc2)
                  t[j][i] = t[i][j]
                  print(t[i][j], end= '')  

In [ ]:
t[0]

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = TfidfVectorizer()
sparse_matrix = count_vectorizer.fit_transform(matched_topic[])

In [99]:
from sklearn.metrics.pairwise import cosine_similarity
a=[]
a= cosine_similarity( sparse_matrix, sparse_matrix)

In [ ]:
a[100]